In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_page(url):
    """Fetch the webpage and return the response object."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        return response
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_laptop_data(product):
    """Extract data for a single laptop from the product container."""
    brand_name = product.find('h4', class_='caption').find('a').text.strip()
    model_number = product.find('p', class_='model').text.strip() if product.find('p', class_='model') else 'N/A'
    price = product.find('p', class_='price').text.strip() if product.find('p', class_='price') else 'N/A'

    # For detailed specs, follow each product link
    product_link = product.find('h4', class_='caption').find('a')['href']
    product_response = fetch_page(product_link)
    
    if product_response:
        product_soup = BeautifulSoup(product_response.content, 'html.parser')
        specs = product_soup.find('div', class_='tab-content')
        
        # Initialize default values
        processor = ram = storage = vga = screen_size = 'N/A'
        
        if specs:
            specs_list = specs.find_all('tr')
            for spec in specs_list:
                title = spec.find('td', class_='text-left').text.strip()
                value = spec.find('td', class_='text-right').text.strip()
                if 'Processor' in title:
                    processor = value
                elif 'RAM' in title:
                    ram = value
                elif 'Storage' in title:
                    storage = value
                elif 'VGA' in title:
                    vga = value
                elif 'Screen Size' in title:
                    screen_size = value

        return {
            'Brand Name': brand_name,
            'Model Number': model_number,
            'Processor': processor,
            'RAM': ram,
            'Storage': storage,
            'VGA': vga,
            'Screen Size': screen_size,
            'Price': price
        }
    return None

def main():
    url = 'https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch'
    response = fetch_page(url)

    if response:
        soup = BeautifulSoup(response.content, 'html.parser')
        products = soup.find_all('div', class_='product-thumb')
        
        laptop_data = []
        for product in products:
            data = extract_laptop_data(product)
            if data:
                laptop_data.append(data)
            time.sleep(1)  # Sleep to avoid overwhelming the server

        # Create a DataFrame and save to CSV
        df = pd.DataFrame(laptop_data)
        df.to_csv('laptops_data.csv', index=False)
        print("Data has been scraped and saved to 'laptops_data.csv'.")

if __name__ == "__main__":
    main()

Data has been scraped and saved to 'laptops_data.csv'.
